In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
data = pd.read_csv('./data/food_affordability.csv')
data.head(10)

,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,...,median_income,affordability_ratio,LL95_affordability_ratio,UL95_affordability_ratio,se_food_afford,rse_food_afford,food_afford_decile,CA_RR_Affordability,ave_fam_size,version
0,757,Food affordability for female-headed household...,2006-2010,1.0,AIAN,CA,6.0,California,NaN,NaN,...,23777.0,0.315779,0.231517,0.400043,0.042991,13.614342,NaN,1.185347,3.34,4/12/2013 4:33
1,757,Food affordability for female-headed household...,2006-2010,2.0,Asian,CA,6.0,California,NaN,NaN,...,38508.0,0.194980,0.183065,0.206895,0.006079,3.117814,NaN,0.731900,3.34,4/12/2013 4:33
2,757,Food affordability for female-headed household...,2006-2010,3.0,AfricanAm,CA,6.0,California,NaN,NaN,...,26192.0,0.286664,0.279661,0.293666,0.003573,1.246349,NaN,1.076054,3.34,4/12/2013 4:33
3,757,Food affordability for female-headed household...,2006-2010,4.0,Latino,CA,6.0,California,NaN,NaN,...,22858.0,0.328475,0.322637,0.334314,0.002979,0.906881,NaN,1.233004,3.34,4/12/2013 4:33
4,757,Food affordability for female-headed household...,2006-2010,5.0,NHOPI,CA,6.0,California,NaN,NaN,...,36737.0,0.204379,0.173762,0.234997,0.015621,7.643255,NaN,0.767183,3.34,4/12/2013 4:33
5,757,Food affordability for female-headed household...,2006-2010,6.0,White,CA,6.0,California,NaN,NaN,...,38641.0,0.194309,0.189570,0.199048,0.002418,1.244406,NaN,0.729381,3.34,4/12/2013 4:33
6,757,Food affordability for female-headed household...,2006-2010,7.0,Multiple,CA,6.0,California,NaN,NaN,...,32866.0,0.228452,0.210008,0.246896,0.009410,4.119149,NaN,0.857543,3.34,4/12/2013 4:33
7,757,Food affordability for female-headed household...,2006-2010,8.0,Other,CA,6.0,California,NaN,NaN,...,30439.0,0.246667,0.176559,0.316775,0.035769,14.501074,NaN,0.925917,3.34,4/12/2013 4:33
8,757,Food affordability for female-headed household...,2006-2010,9.0,Total,CA,6.0,California,NaN,NaN,...,28184.0,0.266403,0.262832,0.269973,0.001821,0.683740,NaN,1.000000,3.34,4/12/2013 4:33
9,757,Food affordability for female-headed household...,2006-2010,1.0,AIAN,CO,6001.0,Alameda,Alameda,6001.0,...,16063.0,0.450857,0.087869,0.813846,0.185198,41.076862,NaN,1.692392,3.21,4/12/2013 4:33
